In [94]:
# !pip install pyinstaller

In [95]:
# !pip install beautifulsoup

In [96]:
# !pip install gspread

In [97]:
# !pip install --upgrade oauth2client

In [98]:
import datetime
import time
from oauth2client.service_account import ServiceAccountCredentials
import gspread

days_ago = datetime.datetime.now() - datetime.timedelta(days=1)
yesterday= days_ago.strftime("%Y-%m-%d")
print(yesterday)

2021-06-15


In [99]:
time1 = datetime.datetime(2021, 6, 16, 9, 0, 0) #기준일 21/06/16
time2 = datetime.datetime.now()
date_gs = (time2-time1).days # 기준날짜와 오늘 날짜의 차
date_gs

0

In [100]:
# !pip install pygsheets

In [101]:
# # 커넥트 플랫폼 자동로그인
# campaign = input("영문 캠페인 명 입력: ")

In [102]:
campaign_list = [6223, 6224]

In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\pc\dev\RPA\chromedriver.exe')
# driver.get("http://admin.mobiconnect.co.kr/admin/login")
# driver.quit()

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1500.55 Safari/537.36


<ipython-input-103-cff64bd2a4de>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\pc\dev\RPA\chromedriver.exe')


In [104]:
driver.get('http://admin.mobiconnect.co.kr/admin/login')
driver.find_element_by_id('userId').send_keys('jmyang')
driver.find_element_by_id('userPwd').send_keys('mobi1109')
driver.find_element_by_xpath('//*[@id="frm"]/li[4]/input').click()

In [105]:
#캠페인 명 검색
# driver.find_element_by_xpath('//*[@id="DataTables_Table_0_filter"]/label/input').send_keys(campaign)

In [106]:
# 캠페인 첫 번째
# driver.implicitly_wait(0.7)
# driver.get('#DataTables_Table_0 > tbody > tr.odd > td.al_left.name_tit > a.tx_blue').click()
# time.sleep(0.5)

In [107]:
for i in campaign_list:
    campaign = i
    
    time.sleep(0.7)
    driver.get('http://admin.mobiconnect.co.kr/admin/adoffers/OfferDetail?offerId='+str(campaign)+'&startDate='+str(yesterday)+'&endDate='+str(yesterday))
    time.sleep(0.7)
    total_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[2]').text
    install_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[4]').text
    event_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[5]').text
    install_data=install_data[:-3]
    event_data = event_data[:-3]
    print(total_data)
    print(install_data)
    print(event_data)
    
    # 구글 시트에 입력 (kb)
    scope = ['https://spreadsheets.google.com/feeds']
    json_file_name = './rpadaily-a0b4f0f12ff6.json'
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
    gc = gspread.authorize(credentials)
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1TaRqRwAbH9NnGR0zl9cynj7l7nmOzSodmqo6HoKJVQA/edit#gid=2124347354'
    
    if campaign == 6224:
        doc = gc.open_by_url(spreadsheet_url)
        sheet1 = doc.worksheet('KBM_able_iOS')
    elif campaign== 6223:
        doc = gc.open_by_url(spreadsheet_url)
        sheet1 = doc.worksheet('KBM_able_AOS')
    else: 
        print("kb캠페인이 아닙니다")
        
    # 기준일(469+ 차이, but 매달 수정해줘야 함)
    sheet1.update_acell('c'+str(469+date_gs), total_data)
    sheet1.update_acell('d'+str(469+date_gs), install_data)
    sheet1.update_acell('e'+str(469+date_gs), event_data)    
    
driver.quit()

1,661,642
6
0
1,490,849
4
1
